### Sagemaker
#### * S3 Buckets - boto3
#### * IAM Roles and Users
#### * AWS Sagemaker - Training and Endpoint creation

In [2]:
import sagemaker
import boto3
from sklearn.model_selection import train_test_split
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\sudha\AppData\Local\sagemaker\sagemaker\config.yaml


In [36]:
sm_boto3 = boto3.client('sagemaker')
session = sagemaker.Session()

region = session.boto_session.region_name

bucket = "sagemakertest2512"

In [37]:
print(f"Using bucket:{bucket} in region:{region}")

Using bucket:sagemakertest2512 in region:us-east-1


In [6]:
df = pd.read_csv("./mob_price_classification_train.csv")

In [7]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [8]:
df['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [25]:
features = list(df.columns)
label = features.pop(-1)

In [28]:
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi']

In [26]:
label

'price_range'

In [29]:
x = df[features]
y = df[label]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [31]:
train_x = pd.DataFrame(X_train, columns=features)
train_x[label] = y_train

In [32]:
test_x = pd.DataFrame(X_test, columns=features)
test_x[label] = y_test

In [33]:
train_x

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
582,1232,0,2.9,1,1,1,24,0.3,169,5,...,361,809,1257,16,10,16,1,0,0,0
159,1840,0,0.5,1,12,0,34,0.7,142,1,...,311,1545,1078,8,0,10,0,0,0,1
1827,1692,0,2.1,0,4,1,2,0.9,106,1,...,1899,1904,3779,9,3,7,1,1,1,3
318,508,0,0.8,0,7,1,42,0.3,94,1,...,39,557,663,13,12,7,1,0,0,0
708,977,1,2.8,1,2,0,35,0.6,165,2,...,1502,1862,3714,19,3,10,0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1224,1,1.6,0,9,0,33,1.0,157,1,...,522,563,3796,10,5,13,1,1,0,3
1216,1158,0,0.7,1,1,1,29,0.7,123,2,...,311,1796,1542,17,9,15,1,0,1,1
1653,1190,0,2.0,1,0,0,40,0.2,93,5,...,1399,1646,3610,13,7,9,0,0,1,3
559,1191,0,2.4,1,2,0,13,0.9,169,1,...,179,1813,1028,14,6,8,1,1,1,0


In [34]:
train_x.to_csv("train_data.csv", index=False)
test_x.to_csv("test_data.csv", index=False)

In [35]:
bucket

'sagemakertest'

In [38]:
## Save data to S3

sk_prefix = "sagemaker/sagemaker-test"
trainpath = session.upload_data("train_data.csv", bucket=bucket, key_prefix=sk_prefix)
testpath = session.upload_data("test_data.csv", bucket=bucket, key_prefix=sk_prefix)

In [39]:
print(f"Train data uploaded to: {trainpath}")
print(f"Test data uploaded to: {testpath}")

Train data uploaded to: s3://sagemakertest2512/sagemaker/sagemaker-test/train_data.csv
Test data uploaded to: s3://sagemakertest2512/sagemaker/sagemaker-test/test_data.csv


### AWS Sagemaker to Train models

In [ ]:
%%writefile script.py


